# Physics Informed FNO for Magnetohydrodynamics Equations

In this notebook, we will study the application of physics informed, data-driven modeling using numerical derivatives, by applying these techniques to the incompressible magnetohydrodynamics (MHD) equations representing an incompressible fluid in the presence of a magnetic field $\mathbf{B}$. In this example, our models will be built using a Fourier Neural Operator (FNO), Tensor Factorized FNO, and trained in conjunction with the PDEs representing our system. 

#### Learning Outcomes
* How to apply physics constraints to neural networks
* Learn how the Fourier Neural Operator can be applied to physics based problems
* Learn how to define PDEs with PhysicsNeMo
* Train PINOs in PhysicsNeMo
* Learn how data driven modeling can help build computationally efficient surrogates for physics problems

## Problem Overview

To examine the properties of PINOs with multiple complex equations, we examined the ability of the networks to reproduce the incompressible magnetohydrodynamics (MHD) equations representing an incompressible fluid in the presence of a magnetic field $\mathbf{B}$. These equations are present in several astrophysical phenomena, including black hole accretion and binary neutron star mergers. 

These equations are given by:

$$\begin{align}
\partial_t \mathbf{u}+\mathbf{u} \cdot \nabla \mathbf{u} &=
-\nabla \left( p+\frac{B^2}{2} \right)/\rho_0 +\mathbf{B}
\cdot \nabla \mathbf{B}+\nu \nabla^2 \mathbf{u}, \\
\partial_t \mathbf{B}+\mathbf{u} \cdot \nabla \mathbf{B} &=
\mathbf{B} \cdot \nabla \mathbf{u}+\eta \nabla^2 \mathbf{B}, \\
\nabla \cdot \mathbf{u} &= 0, \\
\nabla \cdot \mathbf{B} &= 0,
\end{align}$$
 
where $\mathbf{u}$ is the velocity field, $p$ is  the pressure, $B$ is the magnitude of the magnetic field, $\rho_0=1$  is the density of the fluid, $\nu$ is the kinetic viscosity,  and $\eta$ is the magnetic resistivity.  We have two equations  for evolution and two constraint equations.


For the magnetic field divergence equation, we can either include it in the loss function or instead evolve the magnetic vector potential $\mathbf{A}$. This quantity is defined such that

$$\begin{align}
\mathbf{B} = \nabla \times \mathbf{A},
\end{align}$$

which ensures that the divergence of $\mathbf{B}$ is zero. By evolving magnetic vector potential $\mathbf{A}$ instead of the magnetic field $\mathbf{B}$, we have a new evolution equation for the vector potential $\mathbf{A}$. This equation is given by 

$$\begin{align}
\partial_t \mathbf{A} + \mathbf{u} \cdot \nabla \mathbf{A}=\eta \nabla^2 \mathbf{A}.
\end{align}$$

Both scripts to evolve the magnetic field $\mathbf{B}$ or vector potential $\mathbf{A}$ are included.


## Data Creation
To train our model, a representative dataset is first created that gives enough coverage of the solution space to train a surrogate model to make predictions on new data points. To obtain interesting results without additional computational difficulty, we will solve the equations in 2D with periodic boundary conditions. This results in solving a total of 3 evolution PDEs at each time step. Two for the velocity evolution, and one for the magnetic vector potential. 

The solution space to this problem can be obtained numerically by solving the PDEs from above with a numerical solver such as `dedalus`. To generate this data, `dedalus` is used to simulate a 2D periodic incompressible MHD flow with a passive tracer field for visualization. The initial flow is in the x-direction and depends only on z. The problem is non-dimensionalized using the shear-layer spacing and velocity jump, so the resulting viscosity and tracer diffusivity are related to the Reynolds and
Schmidt numbers as:

$$\begin{align}
\nu &= \frac{1}{\text{Re}} \\
\eta &= \frac{1}{\text{Re}_M} \\
D &= \frac{\nu}{\text{Sc}}
\end{align}$$

## Case Setup
Now let's start the problem by importing the required libraries and packages

```python
import torch
import numpy as np
from sympy import Symbol, Eq, Function, Number

import modulus
from modulus.sym.hydra import instantiate_arch , ModulusConfig
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_1d import Line1D
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.monitor import PointwiseMonitor
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.eq.pde import PDE
```


## Defining our Constraints - Setting up the PDE

Constraints are used to define the objectives for training our model. They house a set of nodes form which a computational graph is build for execution as well as the loss function. [PhysicsNemo Sim](https://docs.nvidia.com/deeplearning/physicsnemo/physicsnemo-sym/index.html) provides algorithms and utilities used with PhysicsNeMo core to explicitly physics inform the model training, and provides the means for intuitively setting up multi-objective problems. The types of constraints used will be problem dependent. For this example, we can define the following constraints: 

Data Loss
* Obtain simulation data and compare it to the PINO output.

PDE Loss
* Use the known PDEs of the PINO to describe violations of the time evolution of our system

Constraint Loss
* This loss describes constraints from the PDE. Specifically, the velocity divergence free condition of equation 3 and magnetic divergence free condition of equation 4.

Initial Condition Loss: 
* Input field compared to the output at t=0

Boundary Condition Loss:
* Difference in boundary terms. In our case, we have a periodic boundary constraint. 



To begin setting up our constraints, we can start by defining the MHD equations using the `PDE` class from `physicsnemo.sym.eq.pde`. The process of converting our PDEs into a form that is compatible with `PhysicsNeMo` involves defining a class to hold our equations, called `MHD_PDE`, and including each term of the equations. Each variable of the equations is set up as a `Sympy` `Function`, which is then used to create an attribute of our `MHD_PDE` class that holds the final `equations`. 

Because we have elected to solve the equations in two dimensions, we only have the input variables x, y, t and and the Laplacian. 

In PhysiceNeMo, it is preferable to represent our equations by isolating our target terms on the left, and moving the rest of the equation to the right-hand-side. To do this, various components of each equation are compartmentalized, and the final set of equations is composed from these parts.

Constant magnitude B: https://physics.stackexchange.com/questions/446375/what-are-the-possible-magnetic-fields-with-constant-magnitude



Once all equations are defined, the terms are arranged such that they are represented with the main terms isolated on the left hand side. 

```python
from physicsnemo.sym.eq.pde import PDE
from sympy import Symbol, Function, Number


class MHD_PDE(PDE):
    """MHD PDEs using PhysicsNeMo Sym"""

    name = "MHD_PDE"

    def __init__(self, nu=1e-4, eta=1e-4, rho0=1.0):

        # x, y, time
        x, y, t, lap = Symbol("x"), Symbol("y"), Symbol("t"), Symbol("lap")

        # make input variables
        input_variables = {"x": x, "y": y, "t": t, "lap": lap}

        # make functions
        u = Function("u")(*input_variables)
        v = Function("v")(*input_variables)
        Bx = Function("Bx")(*input_variables)
        By = Function("By")(*input_variables)
        A = Function("A")(*input_variables)
        # pressure
        ptot = Function("ptot")(*input_variables)

        u_rhs = Function("u_rhs")(*input_variables)
        v_rhs = Function("v_rhs")(*input_variables)
        Bx_rhs = Function("Bx_rhs")(*input_variables)
        By_rhs = Function("By_rhs")(*input_variables)
        A_rhs = Function("A_rhs")(*input_variables)

        # initialize constants
        nu = Number(nu)
        eta = Number(eta)
        rho0 = Number(rho0)

        # set equations
        self.equations = {}

        # u · ∇u
        self.equations["vel_grad_u"] = u * u.diff(x) + v * u.diff(y)
        self.equations["vel_grad_v"] = u * v.diff(x) + v * v.diff(y)
        # B · ∇u
        self.equations["B_grad_u"] = Bx * u.diff(x) + v * Bx.diff(y)
        self.equations["B_grad_v"] = Bx * v.diff(x) + By * v.diff(y)
        # u · ∇B
        self.equations["vel_grad_Bx"] = u * Bx.diff(x) + v * Bx.diff(y)
        self.equations["vel_grad_By"] = u * By.diff(x) + v * By.diff(y)
        # B · ∇B
        self.equations["B_grad_Bx"] = Bx * Bx.diff(x) + By * Bx.diff(y)
        self.equations["B_grad_By"] = Bx * By.diff(x) + By * By.diff(y)
        # ∇ × (u × B) = u(∇ · B) - B(∇ · u) B · ∇u − u · ∇B
        self.equations["uBy_x"] = u * By.diff(x) + By * u.diff(x)
        self.equations["uBy_y"] = u * By.diff(y) + By * u.diff(y)
        self.equations["vBx_x"] = v * Bx.diff(x) + Bx * v.diff(x)
        self.equations["vBx_y"] = v * Bx.diff(y) + Bx * v.diff(y)
        # ∇ · B 
        self.equations["div_B"] = Bx.diff(x) + By.diff(y)
        # ∇ · u 
        self.equations["div_vel"] = u.diff(x) + v.diff(y)

        # RHS of MHD equations
        self.equations["u_rhs"] = (
            -self.equations["vel_grad_u"]
            - ptot.diff(x) / rho0
            + self.equations["B_grad_Bx"] / rho0
            + nu * u.diff(lap)
        )
        self.equations["v_rhs"] = (
            -self.equations["vel_grad_v"]
            - ptot.diff(y) / rho0
            + self.equations["B_grad_By"] / rho0
            + nu * v.diff(lap)
        )
        u * By.diff(y) + By * u.diff(y) - v * Bx.diff(y) + Bx * v.diff(y)
        self.equations["Bx_rhs"] = (
            self.equations["uBy_y"] - self.equations["vBx_y"] + eta * Bx.diff(lap)
        )
        self.equations["By_rhs"] = -(
            self.equations["uBy_x"] - self.equations["vBx_x"]
        ) + eta * By.diff(lap)

        self.equations["Du"] = u.diff(t) - u_rhs
        self.equations["Dv"] = v.diff(t) - v_rhs
        self.equations["DBx"] = Bx.diff(t) - Bx_rhs
        self.equations["DBy"] = By.diff(t) - By_rhs

        # Vec potential equations
        self.equations["vel_grad_A"] = u * A.diff(x) + v * A.diff(y)
        self.equations["A_rhs"] = -self.equations["vel_grad_A"] + +eta * A.diff(lap)
        self.equations["DA"] = A.diff(t) - A_rhs
```

Our model's output may then be used to compute the loss between prediction and true values, and computing loss based on initial conditions, PDEs, or data loss using LpLoss.

The specific constraints of our problem 



## Defining our Constraints - Loss Functions 

Now that we have defined our PDE, we can begin to define all of the constraints that make up the loss function for our problem. The loss functions are defined inside of a class called `LossMHD_PhysicsNeMo`, which can use a weighted sum of individual losses for training. 

## Defining our Equations